In [1]:
# https://chenchenhouse.com/goodinfo/
import requests
from bs4 import BeautifulSoup
import sqlite3
import numpy as np
import pandas as pd
import time
# # Google增強DataFrame的渲染和導出
# from google.colab import data_table
# data_table.enable_dataframe_formatter()


# 連接到資料庫A ====================================================================
conn = sqlite3.connect('goodinfoRevenue.db')
# cursor object
cursor = conn.cursor()

sql = '''select * from revenue t'''

# 連接到資料庫B
sqlStock = '''select * from stock t'''
dfStock = pd.read_sql(sqlStock,conn)
dfStock = pd.DataFrame(dfStock)
ids = dfStock['code'][1759:]
# ===================================================================================

count=1759

for id in ids:
  print(id)
  # 爬個股資料
  url = f'https://goodinfo.tw/tw/ShowSaleMonChart.asp?STOCK_ID={id}'
  headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36 Edg/104.0.1293.70'} # goodinfo有擋機器人爬蟲，透過添加headers模仿真實上網的環境就能抓到資料了
  res = requests.get(url, headers = headers,timeout=15)
  res.encoding = "utf-8" # 將編碼設定為【utf-8】，中文字就能顯示出來了
  # res.text

  # BeautifSoup是一個用來解析HTML結構的Python套件，將取回的網頁HTML結構透過提供的方法解析。解析器（html.parser,html5lib,lxml），官方文件lxml為最快
  soup = BeautifulSoup(res.text,"lxml") 
  # select_one：搜索類名、標籤名、id名等，因為我們搜索的是id，在html語言中要加【#】才能搜索到
  data = soup.select_one("#divSaleMonChartDetail")
  # data



  # 【重整表格】 ===============================================================================================
  # prettify()：函數將我們的data物件美化作用
  dfs = pd.read_html(data.prettify())
  df = dfs[1]
  # 網頁的表格是由四格組成，但Python中無法合併單元格一起顯示，所以被合併的表格就會拆分成一格一格顯示
  # 使用columns.get_level_values來取得的最後一行的欄位名
  df.columns = df.columns.get_level_values(2)
  # 刪除所有多於的標題欄
  df2 = df[df["月別"]=="月別"].index
  df2 = df.drop(df2)
  # 重整標題
  # df2.columns = ['月別','開盤','收盤','最高','最低','漲跌(元)','漲跌(%)','月營收(億)','月月增(%)','月年增','累月營收(億)','累月年增','營收(億)','月增(%)','年增(%)','累計營收(億)','累計年增(%)']
  df2.columns = ['date','open','close','high','low','updownYuan','updown','月營收(億)','月月增(%)','月年增','累月營收(億)','累月年增','revenue','mon','yoy','revenueSum','yoySum']

  # 刪除營業收入
  df3 = df2.copy()
  df3.drop(columns=['月營收(億)','月月增(%)','月年增','累月營收(億)','累月年增'],inplace=True)
  # 使用pandas的insert方法，第一个参数指定插入列的位置，第二个参数指定插入列的列名，第三个参数指定插入列的数据
  df3.insert(0,'code',id)
  # =============================================================================================================
  # df3



  # 【添加進資料庫】 =========================================================================================== 
  # for index, row in df3.iterrows():
  #   print(row['代碼'],row['月別'],row['開盤'],row['收盤'],row['最高'],row['最低'],row['漲跌(元)'],row['漲跌(%)'],row['營收(億)'],row['月增(%)'],row['年增(%)'],row['營收(億)'],row['年增(%)'])
  # for index, row in df3.iterrows():
  #   print(row['code'],row['date'],row['open'],row['close'],row['high'],row['low'],row['updownYuan'],row['updown'],row['revenue'],row['mon'],row['yoy'],row['revenueSum'],row['yoySum'])

  for index, row in df3.iterrows():
    try:
      cursor.execute(
      """INSERT OR IGNORE INTO revenue 
          (code,date,open,close,high,low,updownYuan,updown,revenue,mon,yoy,revenueSum,yoySum)
          values(?,?,?,?,?,?,?,?,?,?,?,?,?)""",
          (row['code'],
          row['date'],
          row['open'],
          row['close'],
          row['high'],
          row['low'],
          row['updownYuan'],
          row['updown'],
          row['revenue'],
          row['mon'],
          row['yoy'],
          row['revenueSum'],
          row['yoySum'])
          )
      conn.commit()
    except:
      pass

  count+=1
  print(f'已完成{count}次')
  time.sleep(40)
# =============================================================================================================
# pd.read_sql(sql,conn)
print('爬蟲完畢')


3430
已完成1760次
爬蟲完畢
